In [8]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [9]:
# read data
processed_cleveland <- read_csv("original data/processed.cleveland.data", col_names = F)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [10]:
#rename variables
colnames(processed_cleveland) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", 
                                   "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

processed_cleveland

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1
38,1,3,138,175,0,0,173,0,0.0,1,?,3.0,0


In [11]:
cleveland_filter <- processed_cleveland |>
            select(age, chol, thalach, num) |>
            mutate(num = as_factor(num)) |>
            sample_n(300)
cleveland_filter

age,chol,thalach,num
<dbl>,<dbl>,<dbl>,<fct>
41,306,163,0
58,197,131,0
60,206,132,4
⋮,⋮,⋮,⋮
61,203,161,2
57,274,88,1
53,264,143,0


In [ ]:
#create the TRAIN SET and TEST SET
set.seed(2000)

cleveland_split <- initial_split(cleveland_filter, prop = 0.75, strata = num)
cleveland_train<- training(cleveland_split) 
cleveland_test <- testing(cleveland_split)

# cleveland_vfold <- vfold_cv(cleveland_train, v = 5, strata = num)




In [ ]:
cleveland_recipe <- recipe(num ~ ., data = cleveland_train) |>
                    step_scale(all_predictors()) |>
                    step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

# gridvals = tibble(neighbors = seq(from = 1, to = 80, by = 5))

set.seed(2000)

cleve_workflow <- workflow() |>
            add_recipe(cleveland_recipe) |>
            add_model(knn_spec) |>
            fit(data = cleveland_train)

            # tune_grid(resamples = cleveland_vfold, grid = gridvals) 


In [ ]:
# options(repr.plot.height = 5, repr.plot.width = 6)
# cross_val_plot <- cleve_workflow |>
#                 ggplot(aes(x = neighbors, y = mean)) +
#                 geom_point() +
#                 labs(x = "Neighbors", y = "Accuracy")
# cross_val_plot